In [1]:
import os
import uuid
import faiss
from dotenv import load_dotenv
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.chains import RetrievalQA
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.vectorstores import FAISS
from langchain_community.document_loaders import PyPDFLoader
from time import sleep



In [4]:
loader = PyPDFLoader("https://arxiv.org/pdf/2103.15348.pdf")
pages = loader.load()

In [2]:
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")

data_dir = "data"
loader = PyPDFDirectoryLoader(data_dir)
docs = loader.load_and_split()

# Create unique IDs for each document
for doc in docs:
    doc.metadata["id"] = str(uuid.uuid4())


In [4]:
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")

data_dir = "data"
loader = PyPDFDirectoryLoader(data_dir)
docs = loader.load_and_split()

# Create unique IDs for each document
for doc in docs:
    doc.metadata["id"] = str(uuid.uuid4())

# Initialize the embedding function
embeddings = OpenAIEmbeddings(openai_api_key=api_key, model="text-embedding-ada-002")

In [10]:
from dotenv import load_dotenv
import os
import uuid
import numpy as np
import faiss
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_community.docstore.in_memory import InMemoryDocstore

# Carregar variáveis de ambiente
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")

data_dir = "data"
loader = PyPDFDirectoryLoader(data_dir)
docs = loader.load()

# Criar IDs únicos para cada documento
for doc in docs:
    doc.metadata["id"] = str(uuid.uuid4())

# Inicializar a função de embedding
from langchain_openai import OpenAIEmbeddings
import faiss
import numpy as np
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.vectorstores import FAISS

# Inicializar embeddings
embeddings = OpenAIEmbeddings(openai_api_key=api_key, model="text-embedding-ada-002")
pages = [doc.page_content for doc in docs]
embedded_docs = embeddings.embed_documents(pages)

embedding_dim = len(embedded_docs)
index = faiss.IndexFlatL2(embedding_dim)
print(index.is_trained)
docstore = InMemoryDocstore()
index_to_docstore_id = {i: doc.metadata["id"] for i, doc in enumerate(docs)}

True


In [9]:

# embedded_docs_array = np.array(embedded_docs, dtype=np.float32)
# index.add(embedded_docs_array)

vector_store = FAISS(
    embedding_function=embeddings,
    index=index,
    docstore=docstore,
    index_to_docstore_id=index_to_docstore_id,
)
vector_store.add_documents(docs)


print("There are", len(vector_store.docstore._dict), "docs in the collection")

system_prompt = (
    """You are an assistant for question-answering tasks. 
    Use the following pieces of retrieved context to answer 
    the question. If you don't know the answer, say that you 
    don't know. Use three sentences maximum and keep the 
    answer concise.
    \n\n
    {context}
    """
)
prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("human", "{text}")
])

faiss_retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 3})
rag_chain = RetrievalQA.from_llm(
    llm=ChatOpenAI(openai_api_key=api_key),
    retriever=faiss_retriever,
    return_source_documents=True,
    verbose=True,
)

query = "O que o PDF attention is all you need fala?"
result = rag_chain({"query": query})  # Corrected method call

# Print the result
print(result['result'])  # Access the answer from the result

True


AssertionError: 